In [3]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard

import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
import os
from time import gmtime, strftime
import datetime
from keras.callbacks import TensorBoard
import tensorflow as tf

In [4]:
np.random.seed(1671)  # for reproducibility

# network and training
NB_EPOCH = 400
BATCH_SIZE = 128
VERBOSE = 0
NB_CLASSES = 10 # number of outputs = number of digits
OPTIMIZER = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
N_HIDDEN = 128
VALIDATION_SPLIT=0.2 # how much TRAIN is reserved for VALIDATION
RESHAPED = 784
DROPOUT = 0.3

In [18]:
# data: shuffled and split between train and test sets
#
(X_train, y_train), (X_test, y_test) = mnist.load_data()
#X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
RESHAPED = 28,28,1
#
X_train = X_train.reshape(60000, 28,28,1)
X_test = X_test.reshape(10000, 28,28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalize
#
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

60000 train samples
10000 test samples


In [20]:
# モデルの定義
model = Sequential()

# フィルター
model.add(Conv2D(20, kernel_size=(3,3), padding='same', input_shape=(28,28,1), activation='relu'))
# 出力 28x28x20
# プーリング
model.add(MaxPool2D(pool_size=(2,2)))
# 出力 14x14x20
# フィルター
model.add(Conv2D(50, kernel_size=(5,5), padding='same', activation='relu'))
# 出力 14x14x50
# プーリング
model.add(MaxPool2D(pool_size=(2,2)))
# 出力 7x7x50
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(NB_CLASSES, activation="softmax"))
model.summary()
model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 20)        200       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 20)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 50)        25050     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 50)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 2450)              0         
                                                                 
 dense_6 (Dense)             (None, 500)              

In [21]:
log_dir = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
epochs=0

In [22]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(X_train, Y_train,
batch_size=BATCH_SIZE, epochs=epochs+NB_EPOCH,
initial_epoch=epochs,
callbacks=[tensorboard_callback],
verbose=0, validation_split=VALIDATION_SPLIT)

KeyboardInterrupt: 

In [23]:
score = model.evaluate(X_test, Y_test, verbose=VERBOSE)
print("Test score:", score[0])    # 0.0450
print('Test accuracy:', score[1]) # 0.9925

Test score: 0.045019544661045074
Test accuracy: 0.9925000071525574
